## Test a simple linear regression model

In [1]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

Load in dataset

In [2]:
dbname = 'traffic_police_combined'
username = 'along528'
pswd = 'password'
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [159]:
sql_query = """
SELECT  * FROM combined_rpsi_searches_over_stops_black_over_white;
"""
data = pd.read_sql_query(sql_query,con)
#just look at one year for now
data = data[data['year']==2015]
data = data.reset_index().drop('level_0',1)
data = data.convert_objects(convert_numeric=True)
data = data[data['zipcode']!=27611] #remove nc state highway patrol
data = data.drop(['city','index','state','zipcode','agency','year'],1)
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna()
#with pd.option_context("display.max_rows",100,"display.max_columns", 100):
#    print data[['agency','population']]

/Users/along528/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [180]:
data

population  swnftemp  ftgangoff  ftdrugoff  ftterroff  numcpo  drivhis  cultdiv  white  black  hispanic  asian  nathaw  amerind  multrace  unkrace  male  female  carpat  footpat  cptrnnew  cptrninserv  cpmission  cpplan  cpnone  lessthanplcy  racialprplcy      rpsi
0       144790       120          0          1          0       3        1        0    113      6         1      0       0        0         0        0   104      16       1        1         0            0          1       0       0             1             1  1.561429
1        24399        74          0          6          0       0        1        1     72      2         0      0       0        0         0        0    68       6       1        1         0            0          0       0       0             1             1  3.717454
2        72907       193          2         15          0       7        1        1    172     16         4      0       0        1         0        0   163      30       1        1         1            3          1       0       0             1             1  3.127687
3        45636        44          0          6          0       0        1        1     37      6         1      0       0        0         0        0    40       4       1        0         0            0          0       0       0             1             1  0.928358
4        98966       110          0          8          0      53        1        0    105      5         0      0       0        0         0        0   104       6       1        0         2            1          1       1       0             1             1  1.701112
5        48689       105          0          1          0       2        1        0     91     11         2      0       0        1         0        0    92      13       1        1         0            0          1       0       0             1             1  1.666212
6         9461        17          0          0          0      10        1        0     15      1         1      0       0        0         0        0    15       2       1        0         3            3          1       0       0             1             0  0.555357
7        63154        46          0          4          0      24        1        0     44      2         0      0       0        0         0        0    43       3       1        1         0            0          0       0       0             1             0  0.000000
8       114221       156          0          1          0      12        1        0    143     12         1      0       0        0         0        0   137      19       1        1         1            3          1       0       0             1             1  1.951791
9       155123       119          5          5          0       6        1        0    114      4         0      1       0        0         0        0   102      17       1        0         8            0          0       0       0             1             0  0.987742
..         ...       ...        ...        ...        ...     ...      ...      ...    ...    ...       ...    ...     ...      ...       ...      ...   ...     ...     ...      ...       ...          ...        ...     ...     ...           ...           ...       ...
66       25756        70          0          5          0      11        1        1     65      4         0      1       0        0         0        0    64       6       1        1         1            0          1       0       0             1             0  2.165442
67        1873        17          0          1          0       1        1        1     17      0         0      0       0        0         0        0    16       1       1        1         1            1          1       1       0             1             1  0.981237
68       10463        28          1          1          0      28        1        0     24      4         0      0       0        0         0        0    28       0       1        1         1            0          0       0 

# Build Model

In [4]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [161]:
numeric_categories = ['rpsi','swnftemp','ftdrugoff','ftterroff',
                      'numcpo','white','black','hispanic',
                      'asian','amerind','nathaw','unkrace','multrace',
                      'male','female','population'] #,
                      #'population']
X_int = np.array(data.drop(numeric_categories,1))
enc = OneHotEncoder(sparse=False) #otherwise hard to combined
X_int = enc.fit_transform(X_int)
#X_num = np.array(data[numeric_categories].drop('rpsi',1))
X_num = np.array(data[numeric_categories].drop('rpsi',1))
X_num_scaled = (X_num-np.mean(X_num, axis=0))/np.std(X_num, axis=0)
X = np.concatenate([X_num_scaled,X_int],axis=1)
y = np.array(data['rpsi'])
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [163]:
from sklearn import linear_model
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [179]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((regr.predict(X_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))


('Coefficients: \n', array([ -7.42444132e-01,  -7.13836977e-01,   1.27170759e+00,
         5.77565763e+00,  -9.12261019e-01,  -2.11332952e-01,
        -1.84200749e-01,   1.05133607e+00,   6.90894972e-01,
        -5.81660066e-01,  -1.00240975e-01,  -6.83355347e-01,
        -9.84937501e-01,   6.15331853e-01,   2.04181972e-01,
         7.07387195e+00,   5.32966298e+00,   9.11276262e+00,
         5.20069362e+00,   4.42234400e+00,  -3.85952323e+01,
         7.45589711e+00,  -1.32291358e-14,  -2.97307458e-01,
         2.97307458e-01,  -1.94945601e-14,  -9.51689951e-02,
         9.51689951e-02,   1.22060689e+00,  -4.08747343e-01,
        -1.59710330e+00,  -3.22127099e-01,   1.10737086e+00,
         8.65772728e-01,   1.72361437e+00,  -1.60549732e+00,
        -9.83889780e-01,  -4.89269795e-01,   4.89269795e-01,
         4.89661702e-01,  -4.89661702e-01,   3.12582852e-01,
        -3.12582852e-01,   0.00000000e+00,  -6.88025297e-01,
         6.88025297e-01]))
Residual sum of squares: 20.37
Varian